In [ ]:
import pandas as pd
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
import urllib.request
import zipfile
import os
from keras.models import Sequential
from keras.layers import Embedding,Bidirectional,LSTM,GRU,Dense
import nltk
from nltk.tokenize import word_tokenize
import warnings
import tensorflow as tf
nltk.download('punkt')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [10]:
f=open('train.txt','r')
x_train=[]
y_train=[]
for i in f:
    l=i.split(';')
    y_train.append(l[1].strip())
    x_train.append(l[0])
f=open('test.txt','r')
x_test=[]
y_test=[]
for i in f:
    l=i.split(';')
    y_test.append(l[1].strip())
    x_test.append(l[0])
f=open('val.txt','r')
for i in f:
    l=i.split(';')
    y_test.append(l[1].strip())
    x_test.append(l[0])
data_train=pd.DataFrame({'Text':x_train,'Emotion':y_train})
data_test=pd.DataFrame({'Text':x_test,'Emotion':y_test})
data=data_train.append(data_test,ignore_index=True)

Removing all the prepositions, articles, punctuation marks, stop words, leaving only the important words in the sentences.

In [11]:
def clean_text(data):
    data=re.sub(r"(#[\d\w\.]+)", '', data)
    data=re.sub(r"(@[\d\w\.]+)", '', data)
    data=word_tokenize(data)
    return data
texts=[' '.join(clean_text(text)) for text in data.Text]
texts_train=[' '.join(clean_text(text)) for text in x_train]
texts_test=[' '.join(clean_text(text)) for text in x_test]

tokenizing each sentence, extracting each unique word and createing a dictionary where each unique word is assigned an index.

In [12]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(texts)
sequence_train=tokenizer.texts_to_sequences(texts_train)
sequence_test=tokenizer.texts_to_sequences(texts_test)
index_of_words=tokenizer.word_index
vocab_size=len(index_of_words)+1

The data-set which were obtained has six unique results or emotions, namely: anger, sadness, fear, joy, surprise and love.

Each emotion is given a category value (0–5) in later phases. The 'encoding' dictionary and the 'to categorical' function are used for this reason. We translate the categorical value back to the emotion while trying to get a result.

In [ ]:
num_classes=6
embed_num_dims=300
max_seq_len=500
class_names=['anger','sadness','fear','joy','surprise','love']
X_train_pad=pad_sequences(sequence_train,maxlen=max_seq_len)
X_test_pad=pad_sequences(sequence_test,maxlen=max_seq_len)
encoding={'anger':0,'sadness':1,'fear':2,'joy':3,'surprise':4,'love':5}
y_train=[encoding[x] for x in data_train.Emotion]
y_test=[encoding[x] for x in data_test.Emotion]
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

For the creation of this model, i used a set of word vectors.

Using these word vectors allows us to train our model more quickly and thoroughly, resulting in improved training accuracy.

In [19]:
def create_embedding_matrix(filepath,word_index,embedding_dim):
    vocab_size=len(word_index)+1
    embedding_matrix=np.zeros((vocab_size,embedding_dim))
    with open(filepath) as f:
        for line in f:
            word,*vector=line.split()
            if word in word_index:
                idx=word_index[word]
                embedding_matrix[idx] = np.array(vector,dtype=np.float32)[:embedding_dim]
    return embedding_matrix
fname='word_vectors.vec'
embedd_matrix=create_embedding_matrix(fname,index_of_words,embed_num_dims)

Now I'm working on an architecture for the model's training. To accomplish this, I first developed a **Embedding layer**, whose weights are derived from the word vectors file.


I also added a **Bidirectional layer**, whose features are:
gru_output_size = 128
dropout = 0.2
recurrent_dropout = 0.2


Finally, a **Dense layer** is added which has ‘softmax’ activation.
Adam’s optimizer is used as the optimizer and loss is calculated using ‘categorical_crossentropy’.

*‘model.summary()’ can be used to see the features, layer type, output shape and number of parameters in the model.*

In [20]:
embedd_layer=Embedding(vocab_size,embed_num_dims,input_length=max_seq_len,weights=[embedd_matrix],trainable=False)
gru_output_size=128
bidirectional=True
model=Sequential()
model.add(embedd_layer)
model.add(Bidirectional(GRU(units=gru_output_size,dropout=0.2,recurrent_dropout=0.2)))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

Finally, I train the model using the training set and test the accuracy simultaneously, since the metric of the model is set to ‘accuracy’.

Here, the batch size is taken as 128 and number of epochs is 8. (The batch size and number of epochs can be changed)

In [ ]:
batch_size=128
epochs=8
hist=model.fit(X_train_pad,y_train,batch_size=batch_size,epochs=epochs,validation_data=(X_test_pad,y_test))

After 8 epochs are completed, I wil test the model next.

In [23]:
message=['I am sad.']
seq=tokenizer.texts_to_sequences(message)
padded=pad_sequences(seq,maxlen=max_seq_len)
pred=model.predict(padded)
print('Message:'+str(message))
print('Emotion:',class_names[np.argmax(pred)])

Message:['I am sad.']
Emotion: anger


For the above sentence, I got the result as ‘sadness’, which is most probably correct.

In [25]:
tf.keras.models.save_model(model,'textmodel',overwrite=True,include_optimizer=True,save_format=None,signatures=None,options=None)

INFO:tensorflow:Assets written to: textmodel/assets


INFO:tensorflow:Assets written to: textmodel/assets


The possibilities are unlimited when using this text model to develop a quiz that identifies the user's emotion or perform sentiment analysis of tweets or Reddit posts etc. here in this case it's a mood detecting recipie reccomendation system.